In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv(r"C:\Users\Pratham\Downloads\train(1).csv")


In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
del_cols = ['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked']

In [7]:
df  = df.drop(del_cols,1)

In [8]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch'], dtype='object')

In [9]:
df = df.fillna(df["Age"].mean())

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 41.9+ KB


In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [12]:
df["Sex"] = le.fit_transform(df["Sex"])

In [13]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,1,22.0,1,0
1,1,1,0,38.0,1,0
2,1,3,0,26.0,0,0
3,1,1,0,35.0,1,0
4,0,3,1,35.0,0,0


In [14]:
df.shape

(891, 6)

In [15]:
df.columns = ['output', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch']

In [16]:
df.head()

,output,Pclass,Sex,Age,SibSp,Parch
0,0,3,1,22.0,1,0
1,1,1,0,38.0,1,0
2,1,3,0,26.0,0,0
3,1,1,0,35.0,1,0
4,0,3,1,35.0,0,0


In [17]:
def entropy(col):
    counts = np.unique(col,return_counts=True)
    ent = 0.0
    for ix in counts[1]:
        p = ix/col.shape[0]
        ent += (-1.0*p*np.log2(p))
    return ent

In [18]:
def divide_data(x_data,fkey,fval):
    x_right = pd.DataFrame([],columns=x_data.columns)
    x_left = pd.DataFrame([],columns=x_data.columns)
    for ix in range(x_data.shape[0]):
        val = x_data[fkey].loc[ix]
        if val >=fval:
            x_right = x_right.append(x_data.iloc[ix])
        else:
            x_left = x_left.append(x_data.iloc[ix])
    return x_right,x_left

In [19]:
def information_gain(x_data,fkey,fval):
    right,left = divide_data(x_data,fkey,fval)
    
    l = float(left.shape[0])/x_data.shape[0]
    r = float(right.shape[0])/x_data.shape[0]
    if left.shape[0] == 0 or right.shape[0] == 0:
        return -99999
    i_gain = entropy(x_data.output) - (l * entropy(left.output) + r*entropy(right.output))
    return i_gain
    

In [25]:
class DecisionTree:
    def __init__(self,depth=0,max_depth=5):
        self.left = None
        self.right = None
        self.fkey = None
        self.fval = None
        self.depth = depth
        self.max_depth = max_depth
        self.target = None
    def train(self,x_train):
        features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']
        info_gains = []
        for ix in features:
            i_gain = information_gain(x_train,ix,x_train[ix].mean())
            info_gains.append(i_gain)
        self.fkey = features[np.argmax(info_gains)]
        self.fval = x_train[self.fkey].mean()
        print("Splitting Tree",self.fkey)
        data_right,data_left = divide_data(x_train,self.fkey,self.fval)
        data_right = data_right.reset_index(drop=True)
        data_left = data_left.reset_index(drop=True)
        if data_left.shape[0] == 0 or data_right.shape[0] == 0:
            if x_train.output.mean() >= 0.5:
                self.target = "Positive"
            else:
                self.target = "Negative"
            return
        if self.depth >= self.max_depth:
            if x_train.output.mean() >= 0.5:
                self.target = "Positive"
            else:
                self.target = "Negative"
            return
        self.left = DecisionTree(self.depth+1,self.max_depth)
        self.left.train(data_left)
        self.right = DecisionTree(self.depth+1,self.max_depth)
        self.right.train(data_right)
        if x_train.output.mean() >= 0.5:
            self.target = "Positive"
        else:
            self.target = "Negative"
        return
    def predict(self,test):
        if test[self.fkey] > self.fval:
            if self.right is None:
                return self.target
            return self.right.predict(test)
        if test[self.fkey] < self.fval:
            if self.left is None:
                return self.target
            return self.left.predict(test)

In [26]:
split = int(0.7*df.shape[0])
train_data = df[:split]
test_data = df[split:]
test_data= test_data.reset_index(drop=True)

In [27]:
dt = DecisionTree()

In [28]:
dt.train(train_data)

Splitting Tree Sex
Splitting Tree Pclass
Splitting Tree Age
Splitting Tree SibSp
Splitting Tree Pclass
Splitting Tree Age
Splitting Tree Age
Splitting Tree SibSp
Splitting Tree Parch
Splitting Tree Pclass
Splitting Tree SibSp
Splitting Tree Parch
Splitting Tree Age
Splitting Tree Pclass
Splitting Tree Pclass
Splitting Tree Age
Splitting Tree Age
Splitting Tree Parch
Splitting Tree SibSp
Splitting Tree Age
Splitting Tree Age
Splitting Tree Age
Splitting Tree Age
Splitting Tree SibSp
Splitting Tree SibSp
Splitting Tree SibSp
Splitting Tree Age
Splitting Tree SibSp
Splitting Tree Parch
Splitting Tree Age
Splitting Tree Age
Splitting Tree Age
Splitting Tree Pclass
Splitting Tree Parch
Splitting Tree Pclass
Splitting Tree Age
Splitting Tree Age
Splitting Tree Age
Splitting Tree SibSp
Splitting Tree Age
Splitting Tree Age
Splitting Tree Age
Splitting Tree Age
Splitting Tree Pclass
Splitting Tree SibSp
Splitting Tree SibSp
Splitting Tree Pclass
Splitting Tree Pclass
Splitting Tree Age
Splitti

In [33]:
y_pred = []
for ix in range(test_data.shape[0]):
    y_pred.append(dt.predict(test_data.loc[ix]))

In [34]:
y_pred[:10]

['Negative',
 'Negative',
 'Negative',
 'Negative',
 'Positive',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative']

In [35]:
for i in range(len(y_pred)):
    if y_pred[i] == "Negative":
        y_pred[i] = 0
    else:
        y_pred[i] = 1

In [36]:
np.mean(y_pred == test_data['output'])

0.832089552238806

In [37]:
dt.left.left.fkey

'Age'

In [38]:
from sklearn.tree import DecisionTreeClassifier

In [39]:
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']

In [40]:
sk_tree = DecisionTreeClassifier()

In [41]:
sk_tree.fit(train_data[features],train_data["output"])

DecisionTreeClassifier()

In [42]:
sk_tree.predict(test_data[features])

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 1], dtype=int64)

In [43]:
sk_tree.score(test_data[features],test_data["output"])

0.7910447761194029

In [44]:
from sklearn.ensemble import RandomForestClassifier

In [48]:
rf = RandomForestClassifier(n_estimators=5,criterion="entropy",max_depth=5)

In [49]:
rf.fit(train_data[features],train_data["output"])

RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=5)

In [50]:
rf.score(train_data[features],train_data["output"])

0.8378812199036918

In [51]:
rf.score(test_data[features],test_data["output"])

0.832089552238806

In [53]:
cross_val_score(RandomForestClassifier(n_estimators=5,criterion="entropy",max_depth=5),df[features],df["output"],cv=5).mean()

0.8025233820852427